In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import itertools
import random
import time
import os
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from PIL import Image
import pandas as pd
import cv2
import torch.nn.functional as F
from sklearn.metrics import classification_report, accuracy_score

In [2]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print(f"CUDA is available. Running on {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is available. Running on Quadro P620


In [3]:
import torch
print(torch.cuda.is_available()) 

True


In [4]:
Train_dir = "Museum_Training/Training/"
Test_dir = "Museum_Test/Museum_Validation/"

classification_targets = ['museum-indoor', 'museum-outdoor']

def check_directory(directory):
    if os.path.exists(directory):
        print(f"Directory '{directory}' exists.")
    else:
        print(f"Directory '{directory}' does NOT exist.")

check_directory(Train_dir)
check_directory(Test_dir)

Directory 'Museum_Training/Training/' does NOT exist.
Directory 'Museum_Test/Museum_Validation/' does NOT exist.


In [5]:
# Set device
from sklearn.utils import shuffle
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cuda'
# Define transformations for image preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
     transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load dataset with ImageFolder from 'museum_training' folder
dataset = datasets.ImageFolder(root='../Museum_Training/Training', transform=transform)

# Print class names to verify
print(f"Class names: {dataset.classes}")

# Split dataset into training and testing (80% train, 20% test)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoader for training and testing
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check first batch of labels
for inputs, labels in train_loader:
    print(f"First batch labels: {labels[:10]}")  # Print the first 10 labels in the batch
    break

Class names: ['museum-indoor', 'museum-outdoor']
First batch labels: tensor([0, 0, 1, 0, 0, 0, 0, 1, 0, 0])


In [6]:
print("Number of images:", len(dataset))
print("Train set size:", len(train_dataset))
print("Test set size:", len(test_dataset))
# print("Classes:", dataset.classes)

Number of images: 10000
Train set size: 8000
Test set size: 2000


In [7]:
data_iter = iter(train_loader)
images, labels = next(data_iter)

print("Batch shape:", images.shape)   # Expect: [batch_size, 3, 256, 256]
print("Labels:", labels)

Batch shape: torch.Size([64, 3, 256, 256])
Labels: tensor([1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])


In [8]:
# Check dataset class names
print(f"Class names: {dataset.classes}")

# Check first few labels
for inputs, labels in train_loader:
    print(f"First batch labels: {labels[:10]}")  # Print the first 10 labels in the batch
    break

Class names: ['museum-indoor', 'museum-outdoor']
First batch labels: tensor([1, 1, 0, 1, 1, 1, 1, 1, 1, 1])


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 32 * 32, 512)
        self.fc2 = nn.Linear(512, len(dataset.classes))
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # Define the forward pass
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model and move it to the device
model = CNN().to(device)

In [10]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 20

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    return epoch_loss

# Validation function
def validate(model, test_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    running_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss

            running_loss += loss.item() * images.size(0)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    return epoch_loss, accuracy

In [ ]:
train_losses = []
val_losses = []
val_accuracies = []
best_accuracy = 0.0

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, test_loader, criterion, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    
    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, "
          f"Validation Loss: {val_loss:.4f}, "
          f"Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model

print(f"Best Validation Accuracy: {best_accuracy:.4f}")